<div id="singlestore-header" style="display: flex; background-color: rgba(210, 255, 153, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/chart-network.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Load files from S3 into Shared Tier</h1>
    </div>
</div>

This notebook guides you through data ingestion of CSV files from an AWS S3 location into your shared tier workspace.

# Create a Pipeline from CSV files in AWS S3

In this example, we want to create a pipeline that ingests from a CSV file stored in an AWS S3 bucket. We will guide you through an example with stock market data.

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Make sure to select your database from the drop-down menu at the top of this notebook. It updates the <tt>connection_url</tt> to connect to that database.</p>
    </div>
</div>

## Create a Table

Start by creating a table to store the ingested data. In our example, we will create a `Stocks` table that will store trading data for a specific stock on a given date.

In [1]:
%%sql

CREATE TABLE stocks /* Creating table for sample data. */(
	`date` date NULL,
	`open` double NULL,
	`high` double NULL,
	`low` double NULL,
	`close` double NULL,
	`volume` bigint(20) NULL,
	`Name` text CHARACTER SET utf8 COLLATE utf8_general_ci NULL,
	 SHARD KEY ()
);

## Create a pipeline

We then create a pipeline by pointing the data source to the S3 bucket containing the dataset. In our case, we have a CSV file `all_stocks_5yr.csv` and will be ingesting it into our `Stocks` table via a pipeline `stocks_pipeline`.

To create your own pipeline, you will need the following information:

- The path of the bucket, such as: `'helios-self-poc-stockticker/all_stocks_5yr.csv'`
- The name of the bucket’s region, such as: `us-east-1`
- Your AWS account’s access credentials: `<aws_access_key_id>` and `<aws_secret_access_key>`

*For more on how to retrieve the above information, read our [Pipeline Documentation](https://docs.singlestore.com/cloud/load-data/load-data-with-pipelines/how-to-load-data-using-pipelines/load-data-from-amazon-web-services-aws-s-3/).

In [2]:
%%sql
CREATE OR REPLACE PIPELINE stocks_pipeline /* Creating pipeline for sample data. */
    AS LOAD DATA S3 'helios-self-poc-stockticker/all_stocks_5yr.csv'
    CONFIG '{"region": "us-east-1"}'
    CREDENTIALS '{"aws_access_key_id": "<your_key>",
                  "aws_secret_access_key": "<your_secret>"}'
    BATCH_INTERVAL 45000
    SKIP DUPLICATE KEY ERRORS -- SKIP ALL ERRORS can be used to skip all errors that can be tracked through "Monitor the pipeline for errors"
    INTO TABLE stocks
    FIELDS TERMINATED BY ',' ENCLOSED BY '"' ESCAPED BY '\\'
    LINES TERMINATED BY '\n' STARTING BY ''
    IGNORE 1 LINES
    FORMAT CSV
    (
    	`stocks`.`date`,
    	`stocks`.`open`,
    	`stocks`.`high`,
    	`stocks`.`low`,
    	`stocks`.`close`,
    	`stocks`.`volume`,
    	`stocks`.`Name`
    );

## Start and monitor the pipeline

The CREATE PIPELINE statement creates a new pipeline, but the pipeline has not yet been started, and no data has been loaded. To start a pipeline in the background, run:

In [3]:
%%sql

START PIPELINE stocks_pipeline;

In [4]:
%%sql
show pipelines;

If there is no error or warning, you should see no error message.

In [5]:
%%sql

SELECT * FROM information_schema.pipelines_errors
    WHERE pipeline_name = 'stocks_pipeline';

## Query the table

In [6]:
%%sql

SELECT * FROM stocks LIMIT 5;

## Cleanup resources

In [7]:
%%sql

DROP PIPELINE IF EXISTS test.stocks_pipeline;
DROP TABLE IF EXISTS test.stocks;

<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>